In [26]:
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib.animation as animation
from celluloid import Camera
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from pathlib import Path
import os
import imageio
import itertools



In [27]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [28]:
Nx = 100 + 1;
X = 5;
dx = X/Nx;
x = np.linspace(-X/2,X/2,Nx,endpoint=True)

hi = 1/2j;
# hi = 1

Ny = 64 + 1;
Y = X;
dy = Y/Ny;
y = np.linspace(-Y/2,Y/2,Ny,endpoint=True)

f = 0.5;

In [29]:
x1 , y1 = 0. , -0.
x2 , y2 = -2.7 , 1.4
x3 , y3 = -1.7 , -2.1
x4 , y4 = 3.2 , -1.2
x5 , y5 = 1.55 , 2.33

x_m , y_m = [x1,x2,x3,x4,x5] , [y1,y2,y3,y4,y5]
x_m , y_m = [x1] , [y1]


In [30]:
nxy , nyx = np.meshgrid(y,x)

In [31]:
E_lens = np.exp(1j*(nxy**2 + nyx**2)/(2*f))

In [32]:
E0 =np.zeros([Nx,Ny],dtype=complex)
for ind in range(len(x_m)):
    # x0 , y0 = np.meshgrid(x - x[ind], y - y[ind])
    E_ = np.exp(-((nxy-x_m[ind])**2 + (nyx-y_m[ind])**2)/2)
    E0 += np.copy(E_)

In [33]:
E_s = E0 * E_lens
# E_s = E0

In [34]:
# plt.imshow(np.abs(E_s)**2,extent = (-X/2,X/2,-Y/2,Y/2))
# plt.colorbar()
# plt.show()

In [35]:
Nz = 100
dz = 1./Nz

Ax = np.zeros(Nx) + 1.0
Bx = np.zeros(Nx,dtype=complex) -2.0 * (1 + dx**2/dz/hi)
Cx = np.zeros(Nx) + 1.0

Cx[0] = 0.
Ax[-1] = 0.
Bx[0] = 1.
Bx[-1] = 1.

Ay = np.zeros(Ny) + 1.0
By = np.zeros(Ny,dtype=complex) -2.0 * (1 + dy**2/dz/hi)
Cy = np.zeros(Ny) + 1.0


Cy[0] = 0
Ay[-1] = 0
By[0] = 1
By[-1] = 1


In [36]:
z = 0
E = np.copy(E_s)
Nz = 100
dz = 2*f/Nz
zm = np.linspace(0,2*f,Nz,endpoint=True)



direction1 =f'./{Nx}x{Ny}/'
direction2 = f'{f}/'

Path(f"{direction1}{direction2}").mkdir(parents=True, exist_ok=True)
[os.remove(f'{direction1}{direction2}{file}') for file in os.listdir(f'{direction1}{direction2}') if file.endswith('.png') or file.endswith('.gif')]


Ean = 1/(1 - z/f - 1j*z) * np.exp(-(nxy**2 + nyx**2)/2 *(1+1j*(z - 1/f *(1-z/f)))/((1-z/f)**2+z**2));
Eanprint = np.abs(Ean)**2;

Ep = np.abs(E_s)**2;

fig1 = plt.figure(1,figsize=(9.5,10));
ax11 = fig1.add_subplot(2,2,1,projection='3d')
ax11.plot_surface(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
plt.xlabel('X')
plt.ylabel('Y')
# plt.colorbar();
# axis square;
plt.title(f'Analytic with z={z:0.2f}\n f={f}');

ax12 = fig1.add_subplot(2,2,3)
ims11 = ax12.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
ax12.set_aspect('equal', adjustable='box')
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(ims11);
plt.title(f'Analytic z ={z:0.2f}');
# axis square;
# plt.view(2);

ax13 = fig1.add_subplot(2,2,2,projection='3d')
ax13.plot_surface(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
plt.xlabel('X')
plt.ylabel('Y')
# plt.colorbar();
# axis square;
plt.title(f'z={z:0.2f}\n f={f}');

ax14 = fig1.add_subplot(2,2,4)
ims12 = ax14.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
ax14.set_aspect('equal', adjustable='box')
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(ims12);

filename = f'z={z:0.2f}'
fig1.savefig(f'{direction1}{direction2}{filename}.png')
fig1.clf()
# plt.view(2);
# axis square;




for idz in range(Nz):

    fig1.clf()

    z = z + dz;
    Ean = 1/(1 - z/f - 1j*z) * np.exp(-(nxy**2 + nyx**2)/2 *(1+1j*(z - 1/f *(1-z/f)))/((1-z/f)**2+z**2));
    Eanprint = abs(Ean)**2;

    # цикл по y с дифракцией по x
    for idy in range(Ny):
    # for idy in [Ny//2]:
        Fx = np.zeros(Nx,dtype=complex);
        alphax = np.zeros(Nx,dtype=complex);
        betax = np.zeros(Nx,dtype=complex);
        for idx in range(1,Nx-1):
            Fx[idx] = -E[idx-1,idy] + 2 *(1 - dx**2/dz/hi) * E[idx,idy] - E[idx+1,idy];

        Fx[0] = 0;
        Fx[-1] = 0;

        alphax[1] = -Cx[0]/Bx[0];
        betax[1] = Fx[0]/Bx[0];

        # for i in range(2,Nx-1):
            # alphax[idx+1]= -Cx[idx]/(Ax[idx]*alphax[idx] + Bx[idx]);
            # betax[idx+1] = alphax[idx+1]/Cx[idx]*(Ax(idx)*betax(idx) - Fx(idx));
        '''решение по теории:'''
        for i in range(1,Nx-1):
        #     alphax[i+1] = -Cx[i]/Bx[i] * (1/(1+Ax[i]/Bx[i]*alphax[i]))
        #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
            alphax[i+1]= -Cx[i]/(Ax[i]*alphax[i] + Bx[i]);
            betax[i+1] = alphax[i+1]/Cx[i]*(Ax[i]*betax[i] - Fx[i]);
        #
        '''решение из прошлого семестра'''
        # for i in range(2,Nx):
        #     alphax[i] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i-1])
        #     betax[i] = (Fx[i-1]-Ax[i]*betax[i-1])/(Ax[i]*alphax[i-1]+Bx[i])



        E[-1,idy] = (Fx[-1] - Ax[-1]*betax[-1])/(Ax[-1]*alphax[-1] + Bx[-1]);
        '''по теории'''
        # for idx in range(Nx-1,0,-1):
        #     E[idx-1,idy] = alphax[idx-1] * E[idx,idy] + betax[idx-1]
        '''прошлый семестр'''
        for idx in range(Nx-2 ,-1,-1):
            # print(idx,idy)
            E[idx,idy] = alphax[idx+1] * E[idx+1,idy] + betax[idx+1]
    # figure(2);
    # subplot(2,1,1);
    # surf(nxy,nyx,abs(E).^2,linestyle = ':',edgecolor='none');
    # xlabel('X')
    # ylabel('Y')
    # colorbar();
    # axis square;
    # title({['f=' num2str(f)];['z=' num2str(z)]});
    #
    # subplot(2,1,2)
    # surf(nxy,nyx,abs(E),linestyle = ':',edgecolor='none');
    # xlabel('X')
    # ylabel('Y')
    # colorbar();
    # view(2);
    # axis square;
    ''' цикл по x с дифракцией по y '''
    for idx in range(Nx):
    # for idx in [Nx//2]:
        Fy = np.zeros(Ny,dtype=complex);
        alphay = np.zeros(Ny,dtype=complex);
        betay = np.zeros(Ny,dtype=complex);
        for idy in range(2,Ny-1):
            Fy[idy] = -E[idx,idy-1] + 2 *(1 - dy**2/dz/hi) * E[idx,idy] - E[idx,idy+1];

        Fy[0] = 0;
        Fy[-1] = 0;

        alphay[1] = -Cy[0]/By[0];
        betay[1] = Fy[0]/By[0];

        # for idy in
        #     alphay(idy+1)= -Cy(idy)/(Ay(idy)*alphay(idy) + By(idy));
        #     betay(idy+1) = alphay(idy+1)/Cy(idy)*(Ay(idy)*betay(idy) - Fy(idy));
        '''решение по теории:'''
        for i in range(1,Ny-1):
        #     # alphay[i+1] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i])
        #     # betay[i+1] = (Fy[i-1]-Ay[i]*betay[i])/(Ay[i]*alphay[i]+By[i])
            alphay[i+1]= -Cy[i]/(Ay[i]*alphay[i] + By[i]);
            betay[i+1] = alphay[i+1]/Cy[i]*(Ay[i]*betay[i] - Fy[i]);
        '''решение из прошлого семестра'''
        # for i in range(2,Ny):
        #     alphay[i] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i-1])
        #     betay[i] = (Fy[i-1]-Ay[i]*betay[i-1])/(Ay[i]*alphay[i-1]+By[i])

        E[idx,-1] = (Fy[-1] - Ay[-1]*betay[-1])/(Ay[-1]*alphay[-1] + By[-1]);
        '''по теории'''
        # for idy in range(Ny-1,0,-1):
        #     E[idx,idy-1] = alphay[idy-1] * E[idx,idy] + betay[idy-1];
        '''прошлый семестр'''
        for idy in range(Ny-2,-1,-1):
            E[idx,idy] = alphay[idy+1] * E[idx,idy+1] + betay[idy+1]

            # for i in range(1,Nx):
            #     alphax[i+1] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i])
            #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
            #
            #
            # E[-1,idy] = (Fx[-1] - Ax[-1]*betax[-1])/(Ax[-1]*alphax[-1] + Bx[-1]);
            # for idx in range(Nx-1,-1,-1):
            #     E[idx-1,idy] = alphax[idx] * E[idx,idy] + betax[idx]

    Ep = np.abs(E)**2;

    fig1 = plt.figure(2,figsize=(9.5,10));
    ax21 = fig1.add_subplot(2,2,1,projection='3d')
    ax21.plot_surface(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
    plt.xlabel('X')
    plt.ylabel('Y')
    # plt.colorbar();
    # axis square;
    plt.title(f'Analytic with z={z:0.2f}\n f={f}; step={idz+1}/{Nz}');

    ax22 = fig1.add_subplot(2,2,3)
    ims21 = ax22.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
    ax22.set_aspect('equal', adjustable='box')
    # plt.xlabel('X')
    # plt.ylabel('Y')
    plt.colorbar(ims21);
    plt.title(f'Analytic z ={z:0.2f}');
    # axis square;
    # plt.view(2);

    ax23 = fig1.add_subplot(2,2,2,projection='3d')
    ax23.plot_surface(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
    plt.xlabel('X')
    plt.ylabel('Y')
    # plt.colorbar();
    # axis square;
    plt.title(f'z={z:0.2f}\n f={f}');

    ax24 = fig1.add_subplot(2,2,4)
    ims22 = ax24.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
    ax24.set_aspect('equal', adjustable='box')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(ims22);
    # plt.view(2);
    # axis square;
    filename = f'z={z:0.2f}'
    fig1.savefig(f'{direction1}{direction2}{filename}.png')
    plt.pause(0.01)
    # plt.clf()
# fig1.show(2)


/tmp/ipykernel_59864/2850628894.py:31: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims11 = ax12.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
/tmp/ipykernel_59864/2850628894.py:49: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims12 = ax14.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
/tmp/ipykernel_59864/2850628894.py:186: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same 

In [37]:
N = [100,200,300,400,500]
for x1,x2 in itertools.product(N,repeat=2):
    print(x1,x2)

100 100
100 200
100 300
100 400
100 500
200 100
200 200
200 300
200 400
200 500
300 100
300 200
300 300
300 400
300 500
400 100
400 200
400 300
400 400
400 500
500 100
500 200
500 300
500 400
500 500
